In [6]:
%load_ext autoreload
%autoreload 2

from filemanager import fileset
from cli import cli

fileset_manager = fileset.PDSFileset()

qgis_logs = r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_Filesets\qgis_logs'
filesets_folder = r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_Filesets\pds_filesets'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# pick qgis log
qgis_log_list = cli.search(qgis_logs, '.csv')

Searching *.csv files in directory:\\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_Filesets\qgis_logs
Scanned filesets are: 

0 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_Filesets\qgis_logs\ABP53_UTM34N_MBEStrack.csv
1 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_Filesets\qgis_logs\ABP53_UTM35N_MBEStrack.csv
2 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_Filesets\qgis_logs\ABP53stage03_UTM34N_MBEStrack.csv


In [8]:
qgis_log_picked = cli.pick_from_list(qgis_log_list)

Please type two numbers that define the range of filesets you like to copy or move
Please type the first number: 
Please type the second number: 
Do you want to add more elements? (Y/N)


Picked elements:
ABP53stage03_UTM34N_MBEStrack.csv


In [9]:
import shutil
import os

def qgislogToFilesets(output_dir, qgis_log):
    
    filesets = []
    svp_filesets = []
    filesets_dict = {}

    with open(qgis_log, 'r') as csv_file:
        file_content = csv_file.read().splitlines()

        header = file_content[0].split(';')
        
        filename_idx = 0
        svp_idx = 0
        fileset_idx = 0
        
        for field in header:
            if field == 'Fname' or field == 'fname':
                break
            else:
                filename_idx += 1
        for field in header:
            if field == 'svp_station':
                break
            else:
                svp_idx += 1
                
        for field in header:
            if field == 'fileset':
                break
            else:
                fileset_idx += 1
    
        for num, line in enumerate(file_content[1:]):
            line_content = line.split(';')
            file_name = line_content[filename_idx]
            fileset_name = line_content[fileset_idx]
            svp_name = line_content[svp_idx]

            if fileset_name in filesets_dict.keys():
                filesets_dict[fileset_name].append((file_name, svp_name))
            else:
                filesets_dict[fileset_name] = []
                filesets_dict[fileset_name].append((file_name, svp_name))
    
    for key in filesets_dict.keys():
        fileset_name = key

        fs = fileset.PDSFileset()
        fs.setname(fileset_name)
        
        for filename, svp_name in filesets_dict[key]:
            fs.addfile(filename=filename, svp_name=svp_name)
        filesets.append(fs)
        svp_fs = fs.create_svp_filesets()
        svp_filesets.append(svp_fs)
    
    
    for fs, svpfs in zip(filesets,svp_filesets):
        fs_dir = os.path.join(output_dir,fs.name)
        
        if not os.path.exists(fs_dir):
            os.mkdir(fs_dir)
            
        print(f'{fs.name} - {len(fs.files)}')
        fs.savefileset(dirpath=fs_dir)
        len_svp = 0
        for svfs in svpfs:
            print(f'{svfs.name} - {len(svfs.files)}')
            svfs.savefileset(dirpath=fs_dir)
            len_svp += len(svfs.files)
        print(len_svp)

In [10]:
qgislogToFilesets(filesets_folder, qgis_log_picked[0])

abp53seb03_id007_Taran_S_Po - 89
abp53seb03_id007_Taran_S_Po_ABP53SVP-038 - 20
abp53seb03_id007_Taran_S_Po_ABP53SVP-041 - 8
abp53seb03_id007_Taran_S_Po_ABP53SVP-040 - 33
abp53seb03_id007_Taran_S_Po_ABP53SVP-042 - 12
abp53seb03_id007_Taran_S_Po_ABP53SVP-039 - 15
abp53seb03_id007_Taran_S_Po_ABP53068 - 1
89
abp53seb03_id008_Taran_S_Po - 80
abp53seb03_id008_Taran_S_Po_ABP53SVP-045 - 15
abp53seb03_id008_Taran_S_Po_ABP53SVP-041 - 15
abp53seb03_id008_Taran_S_Po_ABP53SVP-043 - 22
abp53seb03_id008_Taran_S_Po_ABP53SVP-042 - 18
abp53seb03_id008_Taran_S_Po_ABP53SVP-044 - 8
abp53seb03_id008_Taran_S_Po_ABP53SVP-040 - 2
80
abp53seb03_id011_ToSVPStation_Tr - 11
abp53seb03_id011_ToSVPStation_Tr_ABP53SVP-047 - 3
abp53seb03_id011_ToSVPStation_Tr_ABP53070 - 8
11
abp53seb03_id003_ToBaltyisk_Tr - 53
abp53seb03_id003_ToBaltyisk_Tr_ABP53SVP-039 - 21
abp53seb03_id003_ToBaltyisk_Tr_ABP53068 - 20
abp53seb03_id003_ToBaltyisk_Tr_ABP53SVP-038 - 11
abp53seb03_id003_ToBaltyisk_Tr_ABP53SVP-049 - 1
53
abp53seb03_id001_